In [1]:
import json
from ollama import Client
from IPython.display import display
import ipywidgets as widgets
import git
import os
import difflib
from ollama import Message


class GitChatManager:
    def __init__(self, repo_path='./repo', file_name='chat_history.json'):
        self.repo_path = repo_path
        self.file_path = os.path.join(repo_path, file_name)

        if not os.path.exists(repo_path):
            os.makedirs(repo_path)
            self.repo = git.Repo.init(repo_path)
        else:
            try:
                self.repo = git.Repo(repo_path)
            except git.exc.InvalidGitRepositoryError:  # type: ignore
                self.repo = git.Repo.init(repo_path)

        if not os.path.exists(self.file_path):
            initial_content = {
                "messages": [{
                    "role": "user",
                    "content": ""
                }]
            }
            self._save_json(initial_content)
            self.repo.index.add([file_name])
            self.repo.index.commit('Initial commit')

        self.last_content = self._read_current_content()

    def _read_current_content(self):
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as f:
                return f.read()
        return ''

    def _save_json(self, content_dict):
        with open(self.file_path, 'w') as f:
            json.dump(content_dict, f, indent=2)

    def save_and_commit(self, content, author):
        old_content = self.last_content

        if isinstance(content, dict):
            content = json.dumps(content, indent=2)

        with open(self.file_path, 'w') as f:
            f.write(content)

        diff = list(difflib.unified_diff(
            old_content.splitlines(keepends=True),
            content.splitlines(keepends=True),
            fromfile='before',
            tofile='after'
        ))

        if diff:
            self.repo.index.add([os.path.basename(self.file_path)])
            self.repo.index.commit(f'Update from {author}')
            self.last_content = content

        return ''.join(diff)


class ChatInterface:
    def __init__(self):
        self.client = Client(host='http://ollama:11434')
        self.git_manager = GitChatManager()

        self.editor = widgets.Textarea(
            value=self.git_manager._read_current_content(
            ) or '{"messages": [{"role": "user", "content": ""}]}',
            placeholder='Edit JSON here...',
            layout={'width': '100%', 'height': '400px'}
        )

        self.submit_button = widgets.Button(
            description='Send',
            button_style='primary',
            layout={'width': 'auto'}
        )
        self.submit_button.on_click(self._on_submit)

    def _validate_json(self, text):
        try:
            return json.loads(text)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON: {str(e)}")

    def _add_assistant_message(self, json_data):
        """新しいアシスタントメッセージを追加"""
        if isinstance(json_data, str):
            json_data = json.loads(json_data)

        if "messages" not in json_data:
            json_data["messages"] = []

        json_data["messages"].append({
            "role": "assistant",
            "content": ""
        })

        return json_data

    def _update_assistant_response(self, json_data, response):
        """最後のアシスタントメッセージを更新"""
        if isinstance(json_data, str):
            json_data = json.loads(json_data)

        if json_data["messages"] and json_data["messages"][-1]["role"] == "assistant":
            json_data["messages"][-1]["content"] = response

        return json_data

    def _add_next_user_message(self, json_data):
        """次のユーザーメッセージのテンプレートを追加"""
        if isinstance(json_data, str):
            json_data = json.loads(json_data)

        json_data["messages"].append({
            "role": "user",
            "content": ""
        })

        return json_data

    def _on_submit(self, sender):
        try:
            # Validate JSON
            json_data = self._validate_json(self.editor.value)

            # Commit human's changes
            human_diff = self.git_manager.save_and_commit(
                self.editor.value, 'Human')
            if human_diff:
                print("Human's changes:")
                print(human_diff)

            # Add empty assistant message
            json_data = self._add_assistant_message(json_data)
            self.editor.value = json.dumps(json_data, indent=2)

            # Get AI response
            response = ""

            messages = [Message(role=m['role'], content=m['content'])
                        for m in json_data['messages']]
            stream = self.client.chat(
                model='jaahas/gemma-2-9b-it-abliterated',
                messages=messages,
                stream=True
            )

            # Stream AI response
            for chunk in stream:
                if chunk.get('message', {}).get('content'):
                    content = chunk['message']['content']
                    response += content

                    # Update JSON with current response
                    updated_json = self._update_assistant_response(
                        json_data, response)
                    self.editor.value = json.dumps(updated_json, indent=2)

            # Add template for next user message
            final_json = self._add_next_user_message(json_data)
            self.editor.value = json.dumps(final_json, indent=2)

            # Final commit of AI response
            ai_diff = self.git_manager.save_and_commit(
                self.editor.value, 'AI (gemma-2-9b)')
            if ai_diff:
                print("\nAI's response changes:")
                print(ai_diff)

        except Exception as e:
            self.editor.value = f"Error: {str(e)}\n\nPrevious content:\n{self.editor.value}"
            self.git_manager.save_and_commit(self.editor.value, 'Error')

    def display(self):
        display(self.editor)
        display(widgets.HBox([self.submit_button]))


# Create and display interface
chat_interface = ChatInterface()
chat_interface.display()

Textarea(value='{\n  "messages": [\n    {\n      "role": "user",\n      "content": "1 + 1 = ?"\n    },\n    {\…

Human's changes:
--- before
+++ after
@@ -18,7 +18,7 @@
     },
     {
       "role": "user",
-      "content": ""
+      "content": "hello"
     }
   ]
 }

AI's response changes:
--- before
+++ after
@@ -19,6 +19,14 @@
     {
       "role": "user",
       "content": "hello"
+    },
+    {
+      "role": "assistant",
+      "content": "Hello! \ud83d\udc4b  It's nice to hear from you. What can I do for you today? \ud83d\ude0a"
+    },
+    {
+      "role": "user",
+      "content": ""
     }
   ]
 }
